# Workshop 2-1

Regression Exercise

In [0]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import ipywidgets as widget
from ipywidgets import interact, interact_manual
import urllib

rng = np.random.RandomState(1)

In [0]:
for file in ['features_rain.pk','labels_rain.pk','data1.csv','data2.csv','data3.csv']:
    urllib.request.urlretrieve('https://github.com/sunnypwang/mwa_workshop/raw/master/' + file, file)

# Part 1 - Exercise Data

In [0]:
def load_csv(filename, features=['x'], labels=['y']):
    df = pd.read_csv(filename + '.csv')
    x = df[features].to_numpy()
    y = df[labels].to_numpy()
    return x,y

In [0]:
for filename in ['data1','data2','data3']:
    x,y = load_csv(filename)
    plt.title(filename)
    plt.scatter(x, y)
    plt.show()

### TODO1

Describe the relationship between X and Y for each dataset

### Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

def fit_linear(x, y):
    linear_reg = LinearRegression()
    linear_reg.fit(x, y)    
    return linear_reg

def predict(x, model):
    y_pred = model.predict(x)
    return y_pred

def simple_data_wrapper(filename,show_trend=False):
    x,y = load_csv(filename,['x'],['y']) 
    x = x.reshape(-1,1)
    linear_reg = fit_linear(x, y)
    y_pred = predict(x, linear_reg)
    
    print('m =',linear_reg.coef_)
    print('c =',linear_reg.intercept_)
    print('loss =',mse(y, y_pred))
    plt.title(filename)
    plt.scatter(x, y)
    
    if show_trend:
        plt.plot(x, y_pred, '--', color='red')
    plt.show()

In [0]:
interact(simple_data_wrapper, filename=['data1','data2','data3'], show_trend=False)

### TODO2

Why is the regression line in data3 has slightly higher slope than it look? What is the effect of outliers?

# Part 2 - Real Data
<a id='real'></a>

### Precipitation Nowcasting ###

Precipitation nowcasting is the the task of predicting the amount of rainfall in a certain region given some kind of sensor data.  The term nowcasting refers to tasks that try to predict the current or near future conditions (within 6 hours). 

You will be given satellite images in 3 different bands covering a 5 by 5 region from different parts of Thailand. In other words, your input will be a 5x5x3 image. Your task is to predict the amount of rainfal in the center pixel. You will first do the prediction using just a simple fully-connected neural network that view each pixel as different input features.

Since the your input is basically an image, we will then view the input as an image and apply CNN to do the prediction. Finally, we can also add a time component since weather prediction can benefit greatly using previous time frames. Each data point actually contain 5 time steps, so each input data point has a size of 5x5x5x3 (time x height x width x channel), and the output data has a size of 5 (time). You will use this time information when you work with RNNs.

Finally, we would like to thank the Thai Meteorological Department for providing the data for this assignment.

### Data Explanation

The data is an hourly measurement of water vapor in the atmosphere, and two infrared measurements of cloud imagery on a latitude-longitude coordinate. Each measurement is illustrated below as an image. These three features are included as different channels in your input data.

<img src="https://raw.githubusercontent.com/burin-n/pattern-recognition/master/HW4/images/wvapor.png" width="200"> <img src="https://raw.githubusercontent.com/burin-n/pattern-recognition/master/HW4/images/cloud1.png" width="200"> <img src="https://raw.githubusercontent.com/burin-n/pattern-recognition/master/HW4/images/cloud2.png" width="200">

We also provide the hourly precipitation (rainfall) records in the month of June, July, August, September, and October from weather stations spreaded around the country. A 5x5 grid around each weather station at a particular time will be paired with the precipitation recorded at the corresponding station as input and output data. 

**features** 
- dim 0: number of entries
- dim 1,2: a 5x5 grid around rain-measued station
- dim 3: water vapor and two cloud imagenaries 

**labels**
- dim 0: number of entries

## 2.1 Linear Regression

### Load Data

In [0]:
x_train_raw = pickle.load(open('features_rain.pk','rb'))
y_train_raw = pickle.load(open('labels_rain.pk','rb'))
print('x_train shape:',x_train_raw.shape)
print('y_train shape:', y_train_raw.shape)

In [0]:
features = ['vapor','infrared-1','infrared-2']
for i in range(3):
    plt.hist(x_train_raw[:,:,:,i].mean(axis=(1,2)))
    plt.title(features[i])
    plt.show()

In [0]:
import seaborn as sns

sns.heatmap(x_train_raw[0,:,:,0],annot=True,fmt='.2f',square=True,cmap="YlGnBu")
plt.show()

### Linear Regression with single feature

In [0]:
from sklearn.preprocessing import StandardScaler
def normalizer(x):
    return StandardScaler().fit_transform(x)

In [0]:
def real_data_wrapper(feature,show_trend=True,normalize=False):

    x = x_train_raw[:,2,2,feature_list[feature]].reshape(-1,1)
    if normalize:
        x = normalizer(x)
    
    y = y_train_raw
    
    linear_reg = fit_linear(x, y)
    y_pred = predict(x, linear_reg)
    
    print('m =',linear_reg.coef_)
    print('c =',linear_reg.intercept_)
    print('loss =',mse(y, y_pred))

    plt.scatter(x, y)
    if show_trend:
        plt.plot(x, y_pred, '--', color='red')
    plt.xlabel(feature)
    plt.ylabel('rainfall')
    plt.show()

In [0]:
feature_list = {'vapor':0,'infrared-1':1,'infrared-2':2}
feat_idx = widget.widgets.Dropdown(options=feature_list.keys(),description='Feature')
interact_manual(real_data_wrapper,feature=feat_idx)

### Linear Regression with multiple features (multivariable linear regression)

Let's use all three features (vapor, latitude, longitude)

Note that we are now solving a equation beyond 2D space, so we cannot draw a scatter plot as before

In [0]:
x_train = x_train_raw[:,2,2]
y_train = y_train_raw

print('x_train shape:',x_train.shape)
print('y_train shape:', y_train.shape)

In [0]:
linear_reg = fit_linear(x_train, y_train)
y_pred = predict(x_train, linear_reg)

print('m =',linear_reg.coef_)
print('c =',linear_reg.intercept_)
print('loss =',mse(y_train, y_pred))

Let's use the surrounding values in a 5x5 grid as features as well.

Right now there are 5 * 5 * 3 = 75 features

In [0]:
x_train = x_train_raw.reshape(-1,75)
y_train = y_train_raw

print('x_train shape:',x_train.shape)
print('y_train shape:', y_train.shape)

In [0]:
linear_reg = fit_linear(x_train, y_train)
y_pred = predict(x_train, linear_reg)

print('m =',linear_reg.coef_)
print('c =',linear_reg.intercept_)
print('loss =',mse(y_train, y_pred))

Notice that the loss is roughly the same. What does this mean?

### Other Linear Regression

Recall that the loss function of simple linear regression is ordinary least squares (OLS)
$$loss = \sum (y - \hat{y})^2$$

However, OLS is unbiased, meaning that it treats all features with the same level of importance. In reality, you would only focus on most important features and discard unrelated features. OLS also suffer from multicolinearity of the dataset.

Ridge regression is a variant of linear regression that addresses these problems by introducing a penalty term to the loss function. This penalty term will help to reduce the sum of coefficients to reduce the weight of unimportant features. This is known as L2 regularization.
The loss function of ridge regression is given by
$$loss = \sum (y - \hat{y})^2 + \lambda \sum \beta^2 $$

Lasso regression is another variant of linear regression that use L1 regularization as a pernalty term instead.
$$loss = \sum (y - \hat{y})^2 + \lambda \sum |\beta| $$

Elastic Net regression is another variant of linear regression that combine L1 and L2 regularization together as a penalty term. In other words, elastic net method includes the lasso and ridge regression. $r$ is determines the ratio between L1 and L2 regularization.
$$loss = \sum (y - \hat{y})^2 + \big{(}r * \lambda_1\sum |\beta|\big{)} + \big{(}(1-r) * \lambda_2\sum \beta^2\big{)}$$

In [0]:
x_train = x_train_raw.reshape(-1,75)
y_train = y_train_raw

print('x_train shape:',x_train.shape)
print('y_train shape:', y_train.shape)

In [0]:
from sklearn.linear_model import Ridge,Lasso,ElasticNet

def models_wrapper(method='linear',normalize=False):

    x = x_train
    y = y_train
    
    if normalize:
        x = normalizer(x)
    
    if method == 'ridge':
        model = Ridge()
    elif method == 'lasso':
        model = Lasso()
    elif method == 'elastic':
        model = ElasticNet()
    else:
        model = LinearRegression()
        
    print('Model used :',method)
    model.fit(x,y)
    y_pred = predict(x, model)
    
    print('m =',linear_reg.coef_)
    print('c =',linear_reg.intercept_)
    print('loss =',mse(y, y_pred))

In [0]:
interact_manual(models_wrapper,method=['ridge','lasso','elastic','linear'])

<a id='real_logistic'></a>

## 2.2 Logistic Regression

ในส่วนของการทำ Logistic Regression นั้นจะใช้ในโจทย์ประเภท Classification ซึ่งจะใช้ label เป็นคลาสต่างๆ โดยเป้าหมายของ Classification Problem คือ ทำนายว่าจะมีโอกาสเป็นคลาสไหนมากที่สุด แทนที่จะทำนายว่ามีปริมาณเท่าไหร่ ดังนั้นจึงต้องแปลงผลเฉลยจากตัวเลขให้เป็นคลาสที่แตกต่างกัน โดยจะกำหนดให้

เมื่อไม่มีปริมาณน้ำฝนเลย เป็นคลาส 0 (ฝนไม่ตก)

เมื่อมีปริมาณน้ำฝนไม่ว่าเท่าใดก็ตาม เป็นคลาส 1 (ฝนตก)

**ดังนั้นโจทย์ของ Logistic Regression คือ ให้ทำนายว่ามีฝนตกหรือไม่**

In [0]:
x_train = x_train_raw.reshape(-1,75)
y_train = (y_train_raw > 0).astype(int)

print('x_train shape:',x_train.shape)
print('y_train shape:', y_train.shape)

#### Explore Data

In [0]:
def class_hist(y,labels=None):
    n,b = np.histogram(y,bins=2,range=(0.,1.))
    print(n)
    plt.pie(n,labels=labels,autopct='%1.1f%%')

    plt.show()

In [0]:
class_hist(y_train,labels=['not rain', 'rain'])

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import plot_confusion_matrix

def evaluate(y_true,y_pred):
    acc = accuracy_score(y_true,y_pred)
    p = precision_score(y_true,y_pred)
    r = recall_score(y_true,y_pred)
    
    print('Accuracy : {:.3f}'.format(acc))
    print('Precision : {:.3f}'.format(p))
    print('Recall : {:.3f}'.format(r))
    
def logistic_wrapper(penalty='l2',max_iter=100,C=1):
    solver = None
    if penalty == 'l1':
        solver = 'liblinear'
    elif penalty == 'l2':
        solver = 'lbfgs'
    else:
        solver = 'saga'
    
    x = normalizer(x_train)
    y = y_train
    
    print('Running...')
    logistic_reg = LogisticRegression(penalty=penalty,solver=solver,max_iter=max_iter,C=C)
    logistic_reg.fit(x,y)
    
    y_pred = predict(x, logistic_reg)
    evaluate(y, y_pred)
    
    plot_confusion_matrix(logistic_reg, x, y)
    plt.show()

In [0]:
penalty_widget = widget.widgets.Dropdown(options=['l1','l2','elasticnet'],value='l2')
max_iter_widget = widget.widgets.IntSlider(value=100,min=100,max=1000,step=50)
C_widget = widget.widgets.FloatLogSlider(value=1,min=-5,max=0,step=1,readout_format='.0e')
interact_manual(logistic_wrapper,penalty=penalty_widget,max_iter=max_iter_widget,C=C_widget)